# 필요한 모듈 불러오기

In [118]:
from __future__ import print_function # python 2 의 print 함수 기능을 python 3 에서도 사용가능하도록 해주는 모듈
from torch import nn, optim, cuda # cuda : GPU 사용을 위한 모듈
from torch.utils import data # Dataloader 사용을 위한 모듈

# datasets : 여러가지 대표적 데이터셋을 웹에서 불러오는 모듈. CIFAR10 데이터 로드
from torchvision import datasets, transforms

# nn의 다양한 함수가 있는 functional 모듈 불러옴
import torch.nn.functional as F

## batch size 설정 및 GPU 사용 기능 활성화

In [119]:
# Batch Size 설정
batch_size = 50

# cuda(GPU) 사용 가능하다 -> cuda.is_available() == 'True'
# 불가능하다 -> cpu 사용
device = 'cuda' if cuda.is_available() else 'cpu'
print('Training CIFAR10 Model on {:^30}\n{:}'.format(device,"============================================"))

Training CIFAR10 Model on              cuda             


# custom dataset 생성
  훈련용과 테스트용 분리

  CIFAR10 Dataset 을 가져옴
  
  torchvision.dataset.CIFAR10

In [120]:
# CIFAR10 Dataset

# dataset 의 root directory : CIFAR10/processed/training.pt 와 CIFAR10/processed/tst.pt 가 존재

# 훈련용
# train = True -> training.pt 로부터 dataset 생성
# PIL image 를 Tensor type 으로 변환
# download = True -> 인터넷으로부터 dataset 다운로드 받고 root directory 에 넣음
# 이미 다운로드되어 있다면 skip
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=True)

# 테스트용
# train = False -> test.pt 로부터 dataset 생성
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor())

Files already downloaded and verified


# custom dataloader 생성
  훈련용, 테스트용 분리

In [121]:
# input pipeline

# train_loader : train_dataset 을 이용한 custom dataloader
# 효율적인 학습을 위해 (편향적 학습 방지) shuffle=True
train_loader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,num_workers=2)

# test_loader : test_dataset 을 이용한 custom dataloader
# 순차적 test 를 위해 shuffle=False
test_loader = data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False,num_workers=2)

# 학습 위한 class 생성
  view : tensor의 원소의 수는 유지하면서 크기(size)나 모양(shape)을 변경하기 위해 사용

  relu : 입력값이 0보다 작으면 0으로 출력, 0보다 크면 입력값 그대로 출력

In [122]:
class Net(nn.Module):

  def __init__(self):
    # 부모 클래스인 nn.Module 의 __init__ 메서드를 Net 에서도 그대로 적용
    super(Net, self).__init__()

    # 8개 층으로 구성된 모델 생성 (input : 32*32*3, output : 10)
    self.l1=nn.Linear(32*32*3, 32*16*3)
    self.l2=nn.Linear(32*16*3, 32*8*3)
    self.l3=nn.Linear(32*8*3, 32*4*3)
    self.l4=nn.Linear(32*4*3, 32*2*3)
    self.l5=nn.Linear(32*2*3, 30*2*3)
    self.l6=nn.Linear(30*2*3, 30*2)
    self.l7=nn.Linear(30*2, 10*2)
    self.l8=nn.Linear(10*2,10)

  # forward propagation
  def forward(self, x):
        
        # -1은 첫번째 차원은 파이토치 자체적으로 크기(size)나 모양(shape)을 설정한다는 의미
        x = x.view(-1, 32*32*3)  # Flatten the data (n, 3, 32, 32)-> (n, 32*32*3)
        
        # x를 Linear 모델에 각각 넣고 relu함수를 통해 나온 결과값을 x에 저장
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        x = F.relu(self.l5(x))
        x = F.relu(self.l6(x))
        x = F.relu(self.l7(x))
        return self.l8(x)


# Net의 인스턴스 선언, 오차함수에 필요한 클래스 생성, 최적화 클래스 생성


In [123]:
# model : Net class 의 instance
model = Net()

# 모델에서 사용하는 input Tensor들은 input = input.to(device) 을 호출해야함
model.to(device)

# CrossEntropyLoss 
criterion = nn.CrossEntropyLoss()

# model.parameters() : SGD 생성자에 model.parameters()를 호출하면
# 모델의 멤버인 2개의 nn.Linear 모듈의 학습 가능한 매개변수들이 포함됨 (w,b)
# update 과정에서 SGD 알고리즘을 채택 (확률적 경사하강법)
# momentum : momentum factor (default: 0)
# 모멘텀은 경사 하강법에 관성을 더해 주는 것
# 매번 기울기를 구하지만, 가중치를 수정하기전 이전 수정 방향(+,-)를 참고하여 같은 방향으로 일정한 비율만 수정되게 하는 방법
# 수정이 양(+) 방향, 음(-) 방향 순차적으로 일어나는 지그재그 현상이 줄어들고, 
# 이전 이동 값을 고려해여 일정 비율만큼 다음 값을 결정하므로 관성의 효과를 낼 수 있음.
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.5)

# 훈련 함수 정의


In [124]:

def train(epoch):
    
    # model.train() -> 학습 모드
    # 학습을 계속하고 싶다면, model.train() 을 호출하여 학습 모드로 설정해야 함
    model.train()
    
    # batch_idx에 배치의 인덱스가 들어가고
    # data에 CIFAR10 dataset의 transform(img) 부분이 들어가고
    # target에 CIFAR10 dataset의 target_transfor(target) 부분이 들어감
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) # 디바이스에 쑤셔넣기
        
        #optimizer.zero_grad() 를 사용하여 수동으로 변화도 버퍼를 0으로 설정하는 것에 유의해야함
        # 이는 역전파(Backprop) 과정에서 변화도가 누적되기 때문
        optimizer.zero_grad()
        
        # y_pred 부분
        # Net class 의 인스턴스 model 에 data를 집어넣어서 return 된 결과값을 output에 저장
        output = model(data)
        
        # loss 함수 생성_ y_pred 자리에 output이, y 자리에 target이 들어간다
        loss = criterion(output, target)
        
        # loss 값 역전파
        loss.backward()
        
        # 최적화 적용
        optimizer.step()

# test 함수 정의


In [125]:

def test(input):
    
    # model.eval() -> test 모드
    # 추론을 실행하기 전에는 반드시 model.eval() 을 호출하여 드롭아웃 및 배치 정규화를 평가 모드로 설정
    # 이걸 안하면 추론 결과가 일관성 없게 출력됨
    model.eval()

    # test_loss 
    test_loss = 0 # test_loss : 손실 값의 합 0 으로 초기화
    correct = 0 # correct 는 정확도 계산을 위한 tool 이다
    
    # 물론 미니배치 단위로 쪼개지는데,
    # data에 CIFAR10 dataset의 transform(img) 부분이 들어가고
    # target에 CIFAR10 dataset의 target_transfor(target) 부분이 들어감
    for data, target in test_loader:
        data, target = data.to(device), target.to(device) # 디바이스에 쑤셔넣기
        
        # y_pred 부분
        # Net class 의 인스턴스 model 에 data를 집어넣어서 return 된 결과값을 output에 저장
        output = model(data)
        
        # sum up batch loss
        # 손실 값의 합
        test_loss += criterion(output, target).item()
        
        # get the index of the max
        # 최대값의 index return
        # keepdim : 벡터 차원의 유지 여부 결정
        pred = output.data.max(1, keepdim=True)[1]
        
        # view_as : view와 비슷 괄호 안에 들어가는 변수와 tensor size 일치시킴
        # 일치하면 +시킴
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        
    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set {input:2d}: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

# 결과 확인 하는 메인함수


In [126]:
if __name__ == '__main__':
  
    for epoch in range(0, 15):
      
      train(epoch) 
      test(epoch)


Test set  0: Average loss: 0.0420, Accuracy: 1779/10000 (18%)
Test set  1: Average loss: 0.0385, Accuracy: 2663/10000 (27%)
Test set  2: Average loss: 0.0364, Accuracy: 3013/10000 (30%)
Test set  3: Average loss: 0.0352, Accuracy: 3266/10000 (33%)
Test set  4: Average loss: 0.0334, Accuracy: 3843/10000 (38%)
Test set  5: Average loss: 0.0320, Accuracy: 4207/10000 (42%)
Test set  6: Average loss: 0.0316, Accuracy: 4279/10000 (43%)
Test set  7: Average loss: 0.0302, Accuracy: 4485/10000 (45%)
Test set  8: Average loss: 0.0297, Accuracy: 4760/10000 (48%)
Test set  9: Average loss: 0.0292, Accuracy: 4756/10000 (48%)
Test set 10: Average loss: 0.0287, Accuracy: 4867/10000 (49%)
Test set 11: Average loss: 0.0279, Accuracy: 4959/10000 (50%)
Test set 12: Average loss: 0.0278, Accuracy: 5024/10000 (50%)
Test set 13: Average loss: 0.0272, Accuracy: 5202/10000 (52%)
Test set 14: Average loss: 0.0277, Accuracy: 5198/10000 (52%)
